<a href="https://colab.research.google.com/github/lapisco/Yolo-Colab/blob/master/Cloud_Yolo_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GLOBAL CONFIGURATIONS
It must be set before runing the script below



In [0]:
DATAFILE_ID = '1UUScyA913_iWAZsS8wWgG0pAeMflAbZ-'
OUTPUT_FOLDER_ID = '1h6tpmWqPd8nVd-4bc8xNPKehf-JBvm54'
FINETUNING = False
RESUME_TRAINING = True
CHECKPOINT_FILE_ID = '1MSSs76SKJeg7v0oBkf61kDuSGyUF4zxC'
WEIGHTS_FILE_NAME = 'yolo-v2_final.weights'

In [0]:
import os
os.environ['OUTPUT_FOLDER_ID'] = OUTPUT_FOLDER_ID
os.environ['WEIGHTS_FILE_NAME'] = WEIGHTS_FILE_NAME

# **Machine Settings**

##Check Nvidia Driver

In [3]:
!nvidia-smi
!cat /proc/driver/nvidia/version

Mon Nov 11 19:51:10 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

##Check CUDA and cuDNN

In [4]:
!nvcc -V
!cat /usr/include/cudnn.h | grep CUDNN_MAJOR -A 2

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 6
#define CUDNN_PATCHLEVEL 3
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"


# **Yolo Detector**

## Setup Yolo

Clone Yolo and install it

In [5]:
!git clone https://github.com/gbhgit/Cloud-Darknet.git
!mv /content/Cloud-Darknet /content/darknet
!cd /content/darknet/ && make

Cloning into 'Cloud-Darknet'...
remote: Enumerating objects: 10954, done.
remote: Total 10954 (delta 0), reused 0 (delta 0), pack-reused 10954
Receiving objects: 100% (10954/10954), 10.59 MiB | 11.50 MiB/s, done.
Resolving deltas: 100% (7361/7361), done.
mkdir -p obj
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DGPU -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:228:21: warning: unused variable ‘n’ [-Wunused-variable]
                 int n = _write(client, outputbuf, outlen);
                     ^
gcc -Iinclude/ -I3rdpar

In [0]:
if FINETUNING:
  !wget http://pjreddie.com/media/files/darknet19_448.conv.23
  !mv darknet19_448.conv.23 /content/darknet/darknet19_448.conv.23

#!cd /content/darknet/ && ./driveDownload.pl https://drive.google.com/file/d/1-2EoUDQS09dDhQzaDrN6_1kFfdjmquci/view?usp=sharing all.zip
#!cd /content/darknet/data && mv /content/darknet/all.zip /content/darknet/data/all.zip && unzip all.zip
#!rm -rf /content/darknet/data/all.zip

## Download data from drive to Colab



In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

**Authenticate** and create the PyDrive client.


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

*Load* a file by ID and save by the name of `data.zip`




In [0]:
downloaded = drive.CreateFile({'id': DATAFILE_ID})
downloaded.GetContentFile('/content/darknet/data.zip')

In [10]:
!cd darknet && ls | grep *.zip

data.zip


### Setup Data

Clean up cache data (just to be sure)

In [0]:
!rm -rf /content/darknet/data/*

In [12]:
!cd darknet/ && unzip  data.zip -d /content/darknet/data

Archive:  data.zip
   creating: /content/darknet/data/cfg/
  inflating: /content/darknet/data/cfg/obj.names  
  inflating: /content/darknet/data/cfg/test.txt  
  inflating: /content/darknet/data/cfg/train.txt  
  inflating: /content/darknet/data/cfg/yolo-v2.cfg  
  inflating: /content/darknet/data/cfg/obj.data  
   creating: /content/darknet/data/img/
  inflating: /content/darknet/data/img/imagem_19_44:59:0.jpg  
  inflating: /content/darknet/data/img/imagem_19_44:59:1.jpg  
  inflating: /content/darknet/data/img/imagem_19_44:59:2.jpg  
  inflating: /content/darknet/data/img/imagem_19_44:59:3.jpg  
  inflating: /content/darknet/data/img/imagem_19_44:59:3.txt  


In [13]:
!cat /content/darknet/data/cfg/obj.data

classes = 6
train = data/cfg/train.txt
valid = data/cfg/test.txt
names = data/cfg/obj.names
backup = backup


## **Train Yolo**

In [0]:
downloaded = drive.CreateFile({'id': CHECKPOINT_FILE_ID})
downloaded.GetContentFile('/content/darknet/{}'.format(WEIGHTS_FILE_NAME))

In [16]:
# ANCHORS EXPLANATION -> https://github.com/pjreddie/darknet/issues/911:
#!cd /content/darknet/ && ./darknet detector calc_anchors data/obj.data -num_of_clusters 5 -width 11 -height 8

if RESUME_TRAINING:
  !cd /content/darknet/ && ./darknet detector train data/cfg/obj.data data/cfg/yolo-v2.cfg $WEIGHTS_FILE_NAME
else:
  !cd /content/darknet/ && ./darknet detector train data/cfg/obj.data data/cfg/yolo-v2.cfg

yolo-v2
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 max                2x 2/ 2    416 x 416 x  32 ->  208 x 208 x  32 0.006 BF
   2 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   3 max                2x 2/ 2    208 x 208 x  64 ->  104 x 104 x  64 0.003 BF
   4 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   5 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   6 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   7 max                2x 2/ 2    104 x 104 x 128 ->   52 x  52 x 128 0.001 BF
   8 conv    256       3 x 3/ 1     52 x  52 x 128 ->   52 x  52 x 256 1.595 BF
   9 conv    128       1 x 1/ 1     52 x  52 x 256 ->   52 x  52 x 128 0.177 BF
  10 conv    256       3 x 3/ 1     52 x  52 x 128 ->   52 x  52 x 256 1.595 BF
  11 max                2x 2/ 2     52 x  52